In [ ]:
def read_data_for_year(year):
    path = 'Eora26_'+year+'_bp/'  
    t = pd.read_csv(path+'Eora26_'+year+'_bp_T.txt', sep='\t', header=None) #shape = (4915, 4915)
    #     row (import from columns) is source and column (export from rows) is destination

#     fd = pd.read_csv(path+'Eora26_'+year+'_bp_FD.txt', sep='\t', header=None) #shape = (4915, 1140) fd = final demand
#     va = pd.read_csv(path+'Eora26_'+year+'_bp_VA.txt', sep='\t', header=None) #shape = (6, 4915) va = value added, primary input
#     q = pd.read_csv(path+'Eora26_'+year+'_bp_Q.txt', sep='\t', header=None) #shape = (2619, 4915) rows in the satellite block
#     qy = pd.read_csv(path+'Eora26_'+year+'_bp_QY.txt', sep='\t', header=None) #shape = (2619, 1140)
   
    files = [('T', t, True)] #[('FD', fd, True), ('Q', q, False), ('T', t, True), ('VA', va, False)] 
    for i, file in enumerate(files):
        with open('indices/labels_'+file[0]+'.txt') as f:
            labels = f.readlines()
            labels = [" ".join(i.rstrip("\n\t").split("\t")).replace("\n","") for i in labels]
        if file[2]:
            file[1].columns = labels
        else:
            file[1].index = labels
    t.index = t.columns
    t['Year'] = [year]*t.shape[0]
    return t

years = ['1996', '2008', '2015']
all_years = pd.concat([read_data_for_year(year) for year in years])
    
    

In [ ]:
def vals_from_cluster(cluster_ind):
    return np.take(df['Count'].values, np.where(kmeans.labels_==cluster_ind)[0])

def get_min_max(cluster_ind):
    return human_format(vals_from_cluster(cluster_ind).min()) + "-" + human_format(vals_from_cluster(cluster_ind).max())

def human_format(num):
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    # add more suffixes if you need them
    return '%d%s' % (num, ['', 'K', 'M', 'G', 'T', 'P'][magnitude])


df = all_years.copy()
df['location'] = [i.split(" ")[0] for i in df.index]
df['Count'] = df['USA USA Industries Electrical and Machinery'] 
df = df[['Count', 'location', 'Year']].groupby(['location', 'Year']).sum().reset_index()
vals = [[0,i] for i in df['Count'].values] #convert to 2d array
n_clusters=10
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(vals)
min_max_mapping = dict([(i, get_min_max(i)) for i in range(n_clusters)])
df['Count-descrete'] = [min_max_mapping[cluster_ind] for cluster_ind in kmeans.labels_]
df.head()


In [ ]:
import plotly_express as px
px.choropleth(df, locations='location', locationmode='country names', color='Count-descrete', animation_frame='Year')

